In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import io
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout

A:\Programs File\Anaconda\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
A:\Programs File\Anaconda\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
A:\Programs File\Anaconda\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
A:\Programs File\Anaconda\lib\site-packages\tensorboard\compat\tensorflow_stub\dty

In [3]:
#download dataframe
df = pd.read_csv('Combined_News_DJIA.csv')
#preprocessing
#Remove punctuation
data = df.iloc[:,2:27]
data.replace("[^a-zA-Z]"," ",regex=True, inplace=True)
#Rename column name for ease of access, and lowercase words
list1=[i for i in range(25)]
new_Index=[str(i) for i in list1]
data.columns= new_Index
for index in new_Index:
    data[index]=data[index].str.lower()

X=[]
for i in range(0,len(data.index)):
    X.append(' '.join(str(x) for x in data.iloc[i,0:25]))

#Splits data into 70% training, 15% testing, 15% validation
X_train, X_test, y_train, y_test = train_test_split(X, df.Label, test_size=0.3, random_state=1)
X_test, X_val, y_test, y_val = train_test_split(X_train, y_train, test_size=0.5, random_state=1)

In [4]:
tokenizer = Tokenizer(num_words = 6000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1

#padding
maxlen = 500
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

#model
embedding_dim = 50


  #model.compile(optimizer='rmsprop',
              ##loss='categorical_crossentropy',
              ##metrics=['accuracy'])


In [8]:
model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                        output_dim=embedding_dim, 
                        input_length=maxlen))
#model.add(Dropout(0.3))
# model.add(LSTM(256, return_sequences=True, dropout=0.3, recurrent_dropout=0.2))
# model.add(LSTM(256, dropout=0.3, recurrent_dropout=0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(160, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
#training
model.fit(X_train, y_train, epochs=20, verbose=False, validation_data=(X_test, y_test), batch_size=5)

In [10]:
#testing
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy for the StockSentiment Prediction model:  {:.4f}".format(accuracy*100))

Testing Accuracy for the StockSentiment Prediction model:  57.8874
